In [1]:
!pip install torch_geometric torch gdown --quiet

In [ ]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

In [2]:
%cd hackaton/

/home/onyxia/work/DL-Hackathon/hackaton


In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets


In [3]:
!ls -lh datasets

total 16K
drwxr-sr-x 2 onyxia users 4.0K May 25 17:03 A
drwxr-sr-x 2 onyxia users 4.0K May 25 17:03 B
drwxr-sr-x 2 onyxia users 4.0K May 25 17:04 C
drwxr-sr-x 2 onyxia users 4.0K May 25 17:04 D


In [4]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse
import torch.nn.functional as F

# Set the random seed
set_seed()


In [5]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [6]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader),  correct / total

In [7]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [8]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [9]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [10]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [11]:
def get_arguments():
    args = {}
    args['train_path'] = get_user_input("Path to the training dataset (optional)")
    args['test_path'] = get_user_input("Path to the test dataset", required=True)
    args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", type_cast=int)
    args['device'] = get_user_input("Which GPU to use if any", default=1, type_cast=int)
    args['gnn'] = get_user_input("GNN type (gin, gin-virtual, gcn, gcn-virtual)", default='gin')
    args['res'] = get_user_input("Residuals in GNN ? (1 yes, 0 no)", default=0)
    args['drop_ratio'] = get_user_input("Dropout ratio", default=0.0, type_cast=float)
    args['num_layer'] = get_user_input("Number of GNN message passing layers", default=5, type_cast=int)
    args['emb_dim'] = get_user_input("Dimensionality of hidden units in GNNs", default=300, type_cast=int)
    args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['epochs'] = get_user_input("Number of epochs to train", default=10, type_cast=int)
    args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (Noisy CE)", default=1, type_cast=int)
    args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)
    args['forget_rate'] = get_user_input("Forget rate for co teaching", default=0.2, type_cast=float)
    args['pooling'] = get_user_input("type of pooling (sum, mean, max, attention, set2set)", default='mean')
    
    return argparse.Namespace(**args)

In [12]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Path to the training dataset (optional) [None]:  datasets/D/train.json.gz
Path to the test dataset [None]:  datasets/D/test.json.gz
Number of checkpoints to save during training [None]:  30
Which GPU to use if any [1]:  
GNN type (gin, gin-virtual, gcn, gcn-virtual) [gin]:  gcn
Residuals in GNN ? (1 yes, 0 no) [0]:  1
Dropout ratio [0.0]:  0.5
Number of GNN message passing layers [5]:  
Dimensionality of hidden units in GNNs [300]:  
Input batch size for training [32]:  
Number of epochs to train [10]:  30
Baseline mode: 1 (CE), 2 (Noisy CE) [1]:  
Noise probability p (used if baseline_mode=2) [0.2]:  
Forget rate for co teaching [0.2]:  0.4
type of pooling (sum, mean, max, attention, set2set) [mean]:  


Arguments received:
train_path: datasets/D/train.json.gz
test_path: datasets/D/test.json.gz
num_checkpoints: 30
device: 1
gnn: gcn
res: 1
drop_ratio: 0.5
num_layer: 5
emb_dim: 300
batch_size: 32
epochs: 30
baseline_mode: 1
noise_prob: 0.2
forget_rate: 0.4
pooling: mean


In [13]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (1 - torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float().sum(dim=1))
        return (losses * weights).mean()

In [14]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
# Création des deux modèles pour Co-Teaching
def build_model():
    if args.gnn == 'gin':
        return GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False, residual = True if args.res == 1 else False, graph_pooling=args.pooling).to(device)
    elif args.gnn == 'gin-virtual':
        return GNN(gnn_type='gin-virtual', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True, residual = True if args.res == 1 else False, graph_pooling=args.pooling).to(device)
    elif args.gnn == 'gcn':
        return GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False, residual = True if args.res == 1 else False, graph_pooling=args.pooling).to(device)
    elif args.gnn == 'gcn-virtual':
        return GNN(gnn_type='gcn-virtual', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True, residual = True if args.res == 1 else False, graph_pooling=args.pooling).to(device)
    else:
        raise ValueError('Invalid GNN type')

# Initialise les deux modèles
model1 = build_model()
model2 = build_model()

# Deux optimiseurs séparés
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)

# Critère (possiblement bruité)
if args.baseline_mode == 2:
    criterion = NoisyCrossEntropyLoss(args.noise_prob)
else:
    criterion = torch.nn.CrossEntropyLoss()

In [15]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [16]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [17]:
def co_teaching_train(train_loader, model1, model2, optimizer1, optimizer2, criterion, device, forget_rate):
    model1.train()
    model2.train()

    total_loss1, total_loss2 = 0.0, 0.0
    total_correct1, total_correct2 = 0, 0
    total_samples = 0

    for data in train_loader:
        data = data.to(device)
        optimizer1.zero_grad()
        optimizer2.zero_grad()

        logits1 = model1(data)
        logits2 = model2(data)

        loss1 = F.cross_entropy(logits1, data.y, reduction='none')
        loss2 = F.cross_entropy(logits2, data.y, reduction='none')

        # Sélection des indices avec plus faible perte
        num_remember = int((1 - forget_rate) * len(loss1))
        _, ind1_sorted = torch.sort(loss1)
        _, ind2_sorted = torch.sort(loss2)
        ind1_update = ind2_sorted[:num_remember]
        ind2_update = ind1_sorted[:num_remember]

        # Rétropropagation croisée
        final_loss1 = criterion(logits1[ind1_update], data.y[ind1_update])
        final_loss2 = criterion(logits2[ind2_update], data.y[ind2_update])

        final_loss1.backward()
        final_loss2.backward()
        optimizer1.step()
        optimizer2.step()

        total_loss1 += final_loss1.item()
        total_loss2 += final_loss2.item()

        total_correct1 += (logits1.argmax(dim=1) == data.y).sum().item()
        total_correct2 += (logits2.argmax(dim=1) == data.y).sum().item()
        total_samples += data.num_graphs

    acc1 = total_correct1 / total_samples
    acc2 = total_correct2 / total_samples

    # Retourner les losses et accuracies de chaque modèle séparément
    return total_loss1 / len(train_loader), total_loss2 / len(train_loader), acc1, acc2

In [18]:
def get_forget_rate_linear_growth(epoch, max_epochs, target_forget_rate):
    half_epochs = max_epochs // 2
    if epoch < half_epochs:
        return (epoch / half_epochs) * target_forget_rate
    else:
        return target_forget_rate

In [19]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0   

    # Valeurs de suivi
    train_losses_model1, train_losses_model2 = [], []
    train_accuracies_model1, train_accuracies_model2 = [], []
    val_losses, val_accuracies = [], []
    
    # Définir les intervalles de sauvegarde
    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]
    
    target_forget_rate = args.forget_rate if hasattr(args, 'forget_rate') else 0.2

    for epoch in range(num_epochs):
        current_forget_rate = get_forget_rate_linear_growth(epoch, num_epochs, target_forget_rate)

        # Co-teaching training step
        loss1, loss2, acc1, acc2 = co_teaching_train(
            train_loader, model1, model2, optimizer1, optimizer2,
            criterion, device, current_forget_rate
        )
    
        # Évaluer uniquement model1 (ou les deux si tu veux)
        val_loss, val_acc = evaluate(val_loader, model1, device, calculate_accuracy=True)
    
        print(f"Epoch {epoch + 1}/{num_epochs}, "
              f"Forget Rate: {current_forget_rate:.4f}, "
              f"Train Loss Model1: {loss1:.4f}, Train Acc Model1: {acc1:.4f}, "
              f"Train Loss Model2: {loss2:.4f}, Train Acc Model2: {acc2:.4f}, "
              f"Val Acc Model1: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, "
                     f"Forget Rate: {current_forget_rate:.4f}, "
                     f"Train Loss Model1: {loss1:.4f}, Train Acc Model1: {acc1:.4f}, "
                     f"Train Loss Model2: {loss2:.4f}, Train Acc Model2: {acc2:.4f}, "
                     f"Val Acc Model1: {val_acc:.4f}")
    
        train_losses_model1.append(loss1)
        train_accuracies_model1.append(acc1)
        train_losses_model2.append(loss2)
        train_accuracies_model2.append(acc2)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
    
        # Sauvegarde du modèle s'il s’améliore
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model1.state_dict(), checkpoint_path)
            print(f"Best model1 updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses_model1, train_accuracies_model1, os.path.join(logs_folder, "plots_model1"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))


Iterating eval graphs: 100%|██████████| 65/65 [00:15<00:00,  4.08batch/s]
Epoch 1/30, Forget Rate: 0.0000, Train Loss Model1: 1.3880, Train Acc Model1: 0.4393, Train Loss Model2: 1.3775, Train Acc Model2: 0.4515, Val Acc Model1: 0.5058


Epoch 1/30, Forget Rate: 0.0000, Train Loss Model1: 1.3880, Train Acc Model1: 0.4393, Train Loss Model2: 1.3775, Train Acc Model2: 0.4515, Val Acc Model1: 0.5058
Best model1 updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_D_best.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:34<00:00,  1.86batch/s]
Epoch 2/30, Forget Rate: 0.0267, Train Loss Model1: 0.9810, Train Acc Model1: 0.5649, Train Loss Model2: 0.9973, Train Acc Model2: 0.5589, Val Acc Model1: 0.5345


Epoch 2/30, Forget Rate: 0.0267, Train Loss Model1: 0.9810, Train Acc Model1: 0.5649, Train Loss Model2: 0.9973, Train Acc Model2: 0.5589, Val Acc Model1: 0.5345
Best model1 updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_D_best.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:16<00:00,  3.99batch/s]
Epoch 3/30, Forget Rate: 0.0533, Train Loss Model1: 0.7972, Train Acc Model1: 0.6058, Train Loss Model2: 0.7970, Train Acc Model2: 0.6051, Val Acc Model1: 0.6284


...

Epoch 28/30, Forget Rate: 0.4000, Train Loss Model1: 0.0708, Train Acc Model1: 0.7100, Train Loss Model2: 0.0700, Train Acc Model2: 0.7104, Val Acc Model1: 0.6975


Iterating eval graphs: 100%|██████████| 65/65 [00:16<00:00,  4.03batch/s]
Epoch 29/30, Forget Rate: 0.4000, Train Loss Model1: 0.0816, Train Acc Model1: 0.7096, Train Loss Model2: 0.0695, Train Acc Model2: 0.7106, Val Acc Model1: 0.4266


Epoch 29/30, Forget Rate: 0.4000, Train Loss Model1: 0.0816, Train Acc Model1: 0.7096, Train Loss Model2: 0.0695, Train Acc Model2: 0.7106, Val Acc Model1: 0.4266


Iterating eval graphs: 100%|██████████| 65/65 [00:16<00:00,  4.06batch/s]
Epoch 30/30, Forget Rate: 0.4000, Train Loss Model1: 0.0703, Train Acc Model1: 0.7145, Train Loss Model2: 0.0652, Train Acc Model2: 0.7222, Val Acc Model1: 0.6829


Epoch 30/30, Forget Rate: 0.4000, Train Loss Model1: 0.0703, Train Acc Model1: 0.7145, Train Loss Model2: 0.0652, Train Acc Model2: 0.7222, Val Acc Model1: 0.6829


In [20]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

12272

In [21]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

In [22]:
model1.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model1, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

Iterating eval graphs: 100%|██████████| 71/71 [00:16<00:00,  4.35batch/s]

Predictions saved to /home/onyxia/work/DL-Hackathon/hackaton/submission/testset_D.csv


In [25]:
import tarfile
import os

def gzip_folder(folder_path, output_file):
    """
    Compresses an entire folder into a single .tar.gz file.

    Args:
        folder_path (str): Path to the folder to compress.
        output_file (str): Path to the output .gz file.
    """
    with tarfile.open(output_file, "w:gz") as tar:
        tar.add(folder_path, arcname=os.path.basename(folder_path))
    print(f"Folder '{folder_path}' has been compressed into '{output_file}'")

# Example usage
folder_path = "/home/onyxia/work/DL-Hackathon/hackaton/checkpoints"            # Path to the folder you want to compress
output_file = "/home/onyxia/work/DL-Hackathon/hackaton/checkpoints.gz"        # Output .gz file name
gzip_folder(folder_path, output_file)

Folder '/home/onyxia/work/DL-Hackathon/hackaton/checkpoints' has been compressed into '/home/onyxia/work/DL-Hackathon/hackaton/checkpoints.gz'
